# 🐟 Fish Speech Setup and API Server for vast.ai

This notebook will help you set up Fish Speech on vast.ai and start the API server.

## 📋 Prerequisites
- Upload this entire folder to your vast.ai Jupyter environment
- Make sure you have a GPU instance (recommended: RTX 3090 or better)
- Ensure you have at least 10GB of storage space


## 🔧 Step 1: System Setup
Run this cell to install system dependencies and set up the environment:

In [ ]:
# Make scripts executable
!chmod +x setup_vastai.sh start_api_server.sh start_webui.sh

# Run the setup script
!./setup_vastai.sh

## 🚀 Step 2: Test the Installation
Let's verify that everything is working correctly:

In [ ]:
# Activate the virtual environment and test CUDA
import subprocess
import sys

# Activate virtual environment
activate_cmd = "source .venv/bin/activate && python -c 'import torch; print(f\"CUDA available: {torch.cuda.is_available()}\"); print(f\"GPU count: {torch.cuda.device_count()}\"); print(f\"GPU name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else \"N/A\"}\")'"

result = subprocess.run(activate_cmd, shell=True, capture_output=True, text=True, executable='/bin/bash')
print("CUDA Test Results:")
print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

## 🌐 Step 3: Start the API Server
Choose one of the options below:

### Option A: Start API Server in Background
This will start the API server and return control to the notebook:

In [ ]:
import subprocess
import time
import requests

# Start the API server in background
print("🚀 Starting Fish Speech API Server...")
process = subprocess.Popen(['./start_api_server.sh'], 
                          stdout=subprocess.PIPE, 
                          stderr=subprocess.STDOUT, 
                          universal_newlines=True)

# Wait a bit for the server to start
print("⏳ Waiting for server to start...")
time.sleep(30)

# Check if server is running
try:
    response = requests.get('http://localhost:8080/docs', timeout=5)
    if response.status_code == 200:
        print("✅ API Server is running!")
        print("📡 API Documentation: http://localhost:8080/docs")
        print("🔗 External URL: http://[YOUR_VAST_IP]:8080")
    else:
        print("⚠️ Server might still be starting...")
except:
    print("⚠️ Server might still be starting or there's an issue...")

print("\n📝 Server process ID:", process.pid)
print("💡 To stop the server, restart this notebook or run: kill", process.pid)

### Option B: Start Web UI (Alternative)
If you prefer the web interface instead of API:

In [ ]:
# Uncomment and run this if you want the Web UI instead
# !./start_webui.sh

## 🧪 Step 4: Test the API
Let's test the API with a simple request:

In [ ]:
import requests
import json
from IPython.display import Audio
import io

# Test the API
def test_fish_speech_api():
    url = "http://localhost:8080/v1/tts"
    
    data = {
        "text": "Hello! This is Fish Speech running on vast.ai. The setup was successful!",
        "max_new_tokens": 1024,
        "chunk_length": 200,
        "top_p": 0.7,
        "repetition_penalty": 1.2,
        "temperature": 0.7
    }
    
    try:
        print("🎤 Generating speech...")
        response = requests.post(url, json=data, timeout=120)
        
        if response.status_code == 200:
            print("✅ Speech generated successfully!")
            
            # Save the audio
            with open('test_output.wav', 'wb') as f:
                f.write(response.content)
            
            # Play the audio in the notebook
            return Audio('test_output.wav')
        else:
            print(f"❌ Error: {response.status_code} - {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Error testing API: {e}")
        return None

# Run the test
audio_result = test_fish_speech_api()
if audio_result:
    display(audio_result)

## 📊 Step 5: Get Server Information
Check server status and get connection details:

In [ ]:
import requests
import socket

# Get the external IP
try:
    external_ip = requests.get('https://api.ipify.org', timeout=5).text
    print(f"🌐 Your vast.ai external IP: {external_ip}")
    print(f"🔗 API Base URL: http://{external_ip}:8080")
    print(f"📚 API Documentation: http://{external_ip}:8080/docs")
except:
    print("❌ Could not get external IP")

# Check API health
try:
    response = requests.get('http://localhost:8080/docs', timeout=5)
    print(f"\n✅ API Server Status: {'Running' if response.status_code == 200 else 'Not responding'}")
except:
    print("\n❌ API Server Status: Not running")

print("\n📋 Connection Information:")
print("   - Port: 8080")
print("   - Protocol: HTTP")
print("   - Endpoint: /v1/tts")
print("   - Method: POST")
print("\n💡 Use the api_client_example.py for connecting from your application!")